In [28]:
import numpy as np
import pandas as pd
import googlemaps
from geopy.distance import geodesic

#Cargar el csv
df_original = pd.read_csv('./Datos/Mi bici/nomenclatura_2024_09.csv')

#Seleccionar solo id, latitud y longitud
df_estaciones = df_original[['id', 'latitude', 'longitude']]

#Mostrar el dataframe
df_estaciones

,id,latitude,longitude
0,2,20.666378,-103.348820
1,3,20.667228,-103.366000
2,4,20.667690,-103.368252
3,5,20.691847,-103.362549
4,6,20.681158,-103.339363
...,...,...,...
369,386,20.670670,-103.324521
370,387,20.697044,-103.335557
371,388,20.679572,-103.336661
372,389,20.718488,-103.374325


In [ ]:
#Cliente de GoogleMaps con API
gmaps = googlemaps.Client(key='NA')

#Dividir las estaciones en bloques de 10
def dividir_en_bloques(df, tamano_bloque=10):
    return [df[i:i+tamano_bloque] for i in range(0, len(df), tamano_bloque)]

#Bloques de estaciones
bloques = dividir_en_bloques(df_estaciones)

#Función para calcular distancias con GoogleMaps
def calcular_distancias_por_bloque(bloque_origen, bloque_destino):
    origenes = bloque_origen[['latitude', 'longitude']].apply(lambda row: f"{row['latitude']},{row['longitude']}", axis=1).tolist()
    destinos = bloque_destino[['latitude', 'longitude']].apply(lambda row: f"{row['latitude']},{row['longitude']}", axis=1).tolist()

    #Request a Distance Matrix API
    resultado = gmaps.distance_matrix(origenes, destinos, mode="bicycling")

    #Asocia las distancias a los ID
    distancias = []
    for i in range(len(bloque_origen)):
        for j in range(len(bloque_destino)):
            origen_id = bloque_origen.iloc[i]['id']
            destino_id = bloque_destino.iloc[j]['id']
            distancia = resultado['rows'][i]['elements'][j]['distance']['value']  # en metros
            distancias.append([origen_id, destino_id, distancia])

    return distancias

#Lista para distancias
todas_las_distancias = []

#Aplicar la función en los bloques
for i, bloque_origen in enumerate(bloques):
    for j, bloque_destino in enumerate(bloques):
        distancias_bloques = calcular_distancias_por_bloque(bloque_origen, bloque_destino)
        todas_las_distancias.extend(distancias_bloques)

#Crear dataframe
df_distancias = pd.DataFrame(todas_las_distancias, columns=['origen_id', 'destino_id', 'distancia_metros'])

#Confirmación de ejecución
print("Archivo CSV con las distancias generado exitosamente.")

Archivo CSV con las distancias generado exitosamente.


In [31]:
#Convertir las id a enteros
df_distancias['origen_id']=df_distancias['origen_id'].astype(int)
df_distancias['destino_id']=df_distancias['destino_id'].astype(int)

#Eliminar las estaciones repetidas
df_distancias=df_distancias[df_distancias['origen_id'] != df_distancias['destino_id']]

#Agregar las distancias geodesicas
df_distancias['geodesic_dis']=df_distancias.apply(lambda row: geodesic(
                                (df_estaciones.loc[df_estaciones['id'] == row['origen_id'], 'latitude'].values[0],
                                 df_estaciones.loc[df_estaciones['id'] == row['origen_id'], 'longitude'].values[0]),
                                (df_estaciones.loc[df_estaciones['id'] == row['destino_id'], 'latitude'].values[0],
                                 df_estaciones.loc[df_estaciones['id'] == row['destino_id'], 'longitude'].values[0])
                                ).meters, axis=1).astype(int)

#Mostrar el dataframe
df_distancias

,origen_id,destino_id,distancia_metros,geodesic_dis
1,2,3,2113,1792
2,2,4,3357,2030
3,2,5,4484,3161
4,2,6,2522,1910
5,2,8,1979,1652
...,...,...,...,...
139869,389,388,7152,5827
139871,389,390,605,508
139872,390,387,6776,4871
139873,390,388,7816,6139


In [32]:
#Guardar el dataframe en un archivo csv
df_distancias.to_csv('./Guardados/Distancias_GoogleMaps_2024.csv', index=False)